In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 
import numpy as np



### Load Dataframes


In [2]:
df_1km = pd.read_csv('Datasets/Machine Learning/Dataframes/1km_All_Birds_DF.csv')
df_5km = pd.read_csv('Datasets/Machine Learning/Dataframes/5km_All_Birds_DF.csv')
df_10km = pd.read_csv('Datasets/Machine Learning/Dataframes/10km_All_Birds_DF.csv')

### Feature Selection

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


x, y = df_1km.iloc[:, 2:-1], df_1km.iloc[:, [-1]]

# it might be worth plotting some of the features to see their distriutions before and after 
# Separate labels from features first before doing this
x_scaled = scaler.fit_transform(x)



In [4]:
# from sklearn.preprocessing import Normalizer, normalize

# X = normalize(X, norm='max')

# new_df_1km = SelectKBest(chi2, k=20).fit_transform(X, y)
# new_df_1km

### Pre-Process Data
Split the data set into train, test and validation sets